In [6]:
#@title Coarse Grain representation of DNA
## For G-Drive Upload
#from google.colab import drive
#drive.mount('/content/drive/')
#%cd /content/drive/My\ Drive

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


file1 = [];count = 0
phos = [];sugar = [];baseA = [];baseT = [];baseG = [];baseC = []
final_phos = [];final_sugar = [];final_baseA = [];final_baseT = [];final_baseG = [];final_baseC = []

with open("test.pdb") as file:
    for line in file:
        if (line[0:4] == "ATOM"):
            a = line.split()
            file1.append(a)




for line in file1:
    if(line[5] != "1" and line[5] != "56"):
        if(line[2] == "P" or line[2] == "O5'" or line[2] == "OP1" or line[2] == "OP2" or line[2] == "O1P" or line[2] == "O2P"):
            phos.append(float(line[6]))
            phos.append(float(line[7]))
            phos.append(float(line[8]))
            final_phos.append(phos)

            phos = []

    if(line[2] == "C2'"  or line[2] == "C4'" or line[2] == "O4'" or line[2] == "C3'" or line[2] == "C5'" or line[2] == "C1'"):
        sugar.append(float(line[6]))
        sugar.append(float(line[7]))
        sugar.append(float(line[8]))
        final_sugar.append(sugar)
        count+=1
        sugar = []

    if(line[3] == "DA" and (line[2] == "N1" or line[2] =="C2" or line[2] =="N3" or line[2] =="C4" or line[2] =="C5" or line[2]=="C6" or line[2] =="N6" or line[2] =="N7" or line[2] =="C8" or line[2] =="N9")):
        baseA.append(float(line[6]))
        baseA.append(float(line[7]))
        baseA.append(float(line[8]))
        final_baseA.append(baseA)
        baseA = []


    if(line[3] == "DT" and (line[2] == "N1" or line[2]=="C2" or line[2] =="O2" or line[2]=="N3" or line[2]=="C4" or line[2]=="O4" or line[2]=="C5" or line[2]=="C5M" or line[2]=="C6")):
        baseT.append(float(line[6]))
        baseT.append(float(line[7]))
        baseT.append(float(line[8]))
        final_baseT.append(baseT)
        baseT = []


    if(line[3] == "DG" and (line[2]=="N1" or line[2]=="C2" or line[2]=="N2" or line[2]=="N3" or line[2]=="C4" or line[2]=="C5" or line[2]=="C6" or line[2]=="O6" or line[2]=="N7" or line[2]=="C8" or line[2]=="N9")):
        baseG.append(float(line[6]))
        baseG.append(float(line[7]))
        baseG.append(float(line[8]))
        final_baseG.append(baseG)
        baseG = []


    if(line[3]=="DC" and (line[2]=="N1" or line[2]=="C2" or line[2]=="O2" or line[2]=="N3" or line[2]=="C4" or line[2]=="N4" or line[2]=="C5" or line[2]=="C6")):
        baseC.append(float(line[6]))
        baseC.append(float(line[7]))
        baseC.append(float(line[8]))
        final_baseC.append(baseC)
        baseC = []



Functions for CG, Parameters & Test-train split

In [ ]:
def coarse_grain(final_res,n_atom):
    final_cg = []; final_res_cg=[];
    constraintsX=[];constraintsY=[];constraintsZ=[];
    final_constrinX=[];final_constrinY=[];final_constrinZ=[];final_dis = [];final_vec = []
    for i in range(0, len(final_res),n_atom):
        a=0;b=0;c=0
        dis=[];vec=[]
        for j in range(n_atom):

            a+= final_res[i+j][0]
            b+= final_res[i+j][1]
            c+= final_res[i+j][2]

        a = round(a/n_atom,3); b = round(b/n_atom,3); c = round(c/n_atom,3)
        for j in range(n_atom):
            ra = a - final_res[i+j][0]
            rb = b - final_res[i+j][1]
            rc = c - final_res[i+j][2]
            constraintsX.append(ra)
            constraintsY.append(rb)
            constraintsZ.append(rc)
            temp = np.array([ra,rb,rc])
            vec.append(temp)
            dis1 = ((a - final_res[i+j][0])**2 + (b - final_res[i+j][1])**2 + (c - final_res[i+j][2])**2)**(0.5)
            dis.append(dis1)
        final_vec.append(vec)
        final_dis.append(dis)
        final_constrinX.append(constraintsX)
        final_constrinY.append(constraintsY)
        final_constrinZ.append(constraintsZ)
        constraintsX = []; constraintsY = []; constraintsZ = []
        final_cg.append(a); final_cg.append(b); final_cg.append(c)
        final_res_cg.append(final_cg)
        final_cg = []
    return final_res_cg,final_constrinX,final_constrinY,final_constrinZ,final_dis,final_vec

def distance_param(final_constrinX,final_constrinY,final_constrinZ):
    final_constrinX = pd.DataFrame(final_constrinX)
    final_constrinY = pd.DataFrame(final_constrinY)
    final_constrinZ = pd.DataFrame(final_constrinZ)
    final_constrinX_sq = final_constrinX**2
    final_constrinY_sq = final_constrinY**2
    final_constrinZ_sq = final_constrinZ**2

    R = final_constrinX_sq + final_constrinY_sq + final_constrinZ_sq
    Rsqrt = np.sqrt(R)
    return Rsqrt




def test_train(final_res_cg,dist_res,final_res,n_res,final_angle):
    dataX=[]
    for i in range(len(final_res_cg)):
        tmp=[]
        for j in range(n_res):

            tmp.append([final_res_cg[i][0],dist_res[j],final_res_cg[i][1],dist_res[j],final_res_cg[i][2],dist_res[j],final_angle[j]])
        dataX.append(tmp)
    dataX=np.reshape(dataX,(len(dataX),7*n_res))
    dataY=final_res
    return dataX,dataY

## Phosphorous

### Testing p on other data

In [ ]:
test_p = [[37.170,  38.080, 124.130],
[35.260,  39.070, 122.810],
[35.820,  36.500, 122.780],
[38.290,  32.730, 119.740]]

In [ ]:
final_phos_one,final_constrinPX_one,final_constrinPY_one,final_constrinPZ_one,final_disp_one,final_vecp_one = coarse_grain(test_p,n_phos)

In [ ]:
test_p = np.array(test_p);
final_phos_one = np.array(final_phos_one)
final_phos_one_reshaped = np.reshape(test_p, (len(final_phos_one), 3*n_phos))

In [ ]:
one_p = test_train(final_phos_one_reshaped,dist_p,test_p,n_phos,final_angle_p)

In [ ]:
print(one_p)

(array([[ 37.17      ,   0.334947  ,  38.08      ,   0.334947  ,
        124.13      ,   0.334947  ,  87.78503187,  37.17      ,
          1.44695243,  38.08      ,   1.44695243, 124.13      ,
          1.44695243, 122.58655891,  37.17      ,   1.45175314,
         38.08      ,   1.45175314, 124.13      ,   1.45175314,
        117.96015479,  37.17      ,   1.4553798 ,  38.08      ,
          1.4553798 , 124.13      ,   1.4553798 , 107.37685042]]), array([[ 37.17,  38.08, 124.13],
       [ 35.26,  39.07, 122.81],
       [ 35.82,  36.5 , 122.78],
       [ 38.29,  32.73, 119.74]]))


In [ ]:
n_phos=4;n_sugar=6;n_baseA = 10;n_baseT=8;n_baseG=11;n_baseC=8

final_phos_cg,final_constrinPX,final_constrinPY,final_constrinPZ,final_disp,final_vecp = coarse_grain(final_phos,n_phos)

dist_p=np.mean(distance_param(final_constrinPX,final_constrinPY,final_constrinPZ))

final_phos_cg = np.array(final_phos_cg);
final_phos = np.array(final_phos);
final_phos = np.reshape(final_phos, (len(final_phos_cg), 3*n_phos))

In [ ]:
dist_p

1.1722580917244876

## Sugar

In [ ]:
final_sugar_cg,final_constrinSX,final_constrinSY,final_constrinSZ,final_disS,final_vecS = coarse_grain(final_sugar,n_sugar)

dist_S=np.mean(distance_param(final_constrinSX,final_constrinSY,final_constrinSZ))

final_sugar_cg = np.array(final_sugar_cg);
final_sugar = np.array(final_sugar);
final_sugar = np.reshape(final_sugar, (len(final_sugar_cg), 3*n_sugar))

## BaseA

In [ ]:
final_baseA_cg,final_constrinAX,final_constrinAY,final_constrinAZ,final_disA,final_vecA = coarse_grain(final_baseA,n_baseA)

dist_BA=np.mean(distance_param(final_constrinAX,final_constrinAY,final_constrinAZ))

final_baseA_cg = np.array(final_baseA_cg);
final_baseA = np.array(final_baseA);
final_baseA = np.reshape(final_baseA, (len(final_baseA_cg), 3*n_baseA))

##  BaseT

In [ ]:
final_baseT_cg,final_constrinTX,final_constrinTY,final_constrinTZ,final_disT,final_vecT = coarse_grain(final_baseT,n_baseT)

dist_BT=np.mean(distance_param(final_constrinTX,final_constrinTY,final_constrinTZ))

final_baseT_cg = np.array(final_baseT_cg);
final_baseT = np.array(final_baseT);
final_baseT = np.reshape(final_baseT, (len(final_baseT_cg), 3*n_baseT))

## BaseG

In [ ]:
final_baseG_cg,final_constrinGX,final_constrinGY,final_constrinGZ,final_disT,final_vecG = coarse_grain(final_baseG,n_baseG)

dist_BG=np.mean(distance_param(final_constrinGX,final_constrinGY,final_constrinGZ))

final_baseG_cg = np.array(final_baseG_cg);
final_baseG = np.array(final_baseG);
final_baseG = np.reshape(final_baseG, (len(final_baseG_cg), 3*n_baseG))

## BaseC

In [ ]:
final_baseC_cg,final_constrinCX,final_constrinCY,final_constrinCZ,final_disC,final_vecC = coarse_grain(final_baseC,n_baseC)

dist_BC=np.mean(distance_param(final_constrinCX,final_constrinCY,final_constrinCZ))

final_baseC_cg = np.array(final_baseC_cg);
final_baseC = np.array(final_baseC);
final_baseC = np.reshape(final_baseC, (len(final_baseC_cg), 3*n_baseC))

In [ ]:

import keras as k
from keras.models import Sequential
from keras.layers import Dense


###  Angles between Phosphorous atoms

In [ ]:
import numpy as np
import vg
import math

final_angle = []
for i in (final_vecp):
    angle = []
    angle.append((vg.angle(i[0], i[1])))
    angle.append((vg.angle(i[1], i[2])))
    angle.append((vg.angle(i[2], i[3])))
    angle.append((vg.angle(i[3], i[0])))
    final_angle.append(angle)

In [ ]:
final_angle_p = np.mean(pd.DataFrame(final_angle))

In [ ]:
final_phos_cg[0]

array([35.215, 22.175, 22.558])

In [ ]:
print("\n\n *********** Training Model for Phosphorus *********** \n")
dataXp, dataYp = test_train(final_phos_cg,dist_p,final_phos,n_phos,final_angle_p)
np_output = len(dataYp[0]);np_input = len(dataXp[0])
Xp_train, Xp_test, yp_train, yp_test = train_test_split(dataXp, dataYp, test_size=0.1, random_state=1)





 *********** Training Model for Phosphorus *********** 



In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(Xp_train)
X_test_normalized = scaler.transform(Xp_test)

In [ ]:
(X_train_normalized).shape,X_test_normalized.shape

((97, 28), (11, 28))

##  Angles between Sugar atoms

In [ ]:
import numpy as np
import vg
import math

final_angle_sugar = []
for i in (final_vecS):
    angle = []
    angle.append((vg.angle(i[0], i[1])))
    angle.append((vg.angle(i[1], i[2])))
    angle.append((vg.angle(i[2], i[3])))
    angle.append((vg.angle(i[3], i[4])))
    angle.append((vg.angle(i[4], i[5])))
    angle.append((vg.angle(i[5], i[0])))

    final_angle_sugar.append(angle)

In [ ]:
final_angle_s = np.mean((pd.DataFrame(final_angle_sugar)))

In [ ]:
print("\n\n *********** Training Model for Sugar *********** \n")
dataXs, dataYs = test_train(final_sugar_cg,dist_S,final_sugar,n_sugar,final_angle_s)
ns_output = len(dataYs[0]);ns_input = len(dataXs[0])
Xs_train, Xs_test, ys_train, ys_test = train_test_split(dataXs, dataYs, test_size=0.1, random_state=1)




 *********** Training Model for Sugar *********** 



In [ ]:
import numpy as np
import vg
import math

final_angle_baseA = []
for i in (final_vecA):
    angle = []
    angle.append((vg.angle(i[0], i[1])))
    angle.append((vg.angle(i[1], i[2])))
    angle.append((vg.angle(i[2], i[3])))
    angle.append((vg.angle(i[3], i[4])))
    angle.append((vg.angle(i[4], i[5])))
    angle.append((vg.angle(i[5], i[6])))
    angle.append((vg.angle(i[6], i[7])))
    angle.append((vg.angle(i[7], i[8])))
    angle.append((vg.angle(i[8], i[9])))
    angle.append((vg.angle(i[9], i[0])))
    final_angle_baseA.append(angle)

In [ ]:
final_angle_baseA = np.mean((pd.DataFrame(final_angle_baseA)))

In [ ]:
print("\n\n *********** Training Model for BaseA *********** \n")
dataXA, dataYA = test_train(final_baseA_cg,dist_BA,final_baseA,n_baseA,final_angle_baseA)
nA_output = len(dataYA[0]);nA_input = len(dataXA[0])
XA_train, XA_test, yA_train, yA_test = train_test_split(dataXA, dataYA, test_size=0.1, random_state=1)




 *********** Training Model for BaseA *********** 



In [ ]:
import numpy as np
import vg
import math

final_angle_baseT = []
for i in (final_vecT):
    angle = []
    angle.append((vg.angle(i[0], i[1])))
    angle.append((vg.angle(i[1], i[2])))
    angle.append((vg.angle(i[2], i[3])))
    angle.append((vg.angle(i[3], i[4])))
    angle.append((vg.angle(i[4], i[5])))
    angle.append((vg.angle(i[5], i[6])))
    angle.append((vg.angle(i[6], i[7])))
    angle.append((vg.angle(i[7], i[0])))


    final_angle_baseT.append(angle)

In [ ]:
final_angle_baseT = np.mean((pd.DataFrame(final_angle_baseT)))

In [ ]:
print("\n\n *********** Training Model for BaseT *********** \n")
dataXT, dataYT = test_train(final_baseT_cg,dist_BT,final_baseT,n_baseT,final_angle_baseT)
nT_output = len(dataYT[0]);nT_input = len(dataXT[0])
XT_train, XT_test, yT_train, yT_test = train_test_split(dataXT, dataYT, test_size=0.1, random_state=1)




 *********** Training Model for BaseT *********** 



In [ ]:
import numpy as np
import vg
import math

final_angle_baseG = []
for i in (final_vecG):
    angle = []
    angle.append((vg.angle(i[0], i[1])))
    angle.append((vg.angle(i[1], i[2])))
    angle.append((vg.angle(i[2], i[3])))
    angle.append((vg.angle(i[3], i[4])))
    angle.append((vg.angle(i[4], i[5])))
    angle.append((vg.angle(i[5], i[6])))
    angle.append((vg.angle(i[6], i[7])))
    angle.append((vg.angle(i[7], i[8])))
    angle.append((vg.angle(i[8], i[9])))
    angle.append((vg.angle(i[9], i[10])))
    angle.append((vg.angle(i[10], i[0])))
    final_angle_baseG.append(angle)

In [ ]:
final_angle_baseG = np.mean((pd.DataFrame(final_angle_baseG)))

In [ ]:
print("\n\n *********** Training Model for BaseG *********** \n")
dataXG, dataYG = test_train(final_baseG_cg,dist_BG,final_baseG,n_baseG,final_angle_baseG)
nG_output = len(dataYG[0]);nG_input = len(dataXG[0])
XG_train, XG_test, yG_train, yG_test = train_test_split(dataXG, dataYG, test_size=0.1, random_state=1)




 *********** Training Model for BaseG *********** 



In [ ]:
import numpy as np
import vg
import math

final_angle_baseC = []
for i in (final_vecC):
    angle = []
    angle.append((vg.angle(i[0], i[1])))
    angle.append((vg.angle(i[1], i[2])))
    angle.append((vg.angle(i[2], i[3])))
    angle.append((vg.angle(i[3], i[4])))
    angle.append((vg.angle(i[4], i[5])))
    angle.append((vg.angle(i[5], i[6])))
    angle.append((vg.angle(i[6], i[7])))
    angle.append((vg.angle(i[7], i[0])))

    final_angle_baseC.append(angle)

In [ ]:
final_angle_baseC = np.mean((pd.DataFrame(final_angle_baseC)))

In [ ]:
print("\n\n *********** Training Model for BaseC *********** \n")
dataXC, dataYC = test_train(final_baseC_cg,dist_BC,final_baseC,n_baseC,final_angle_baseC)
nC_output = len(dataYC[0]);nC_input = len(dataXC[0])
XC_train, XC_test, yC_train, yC_test = train_test_split(dataXC, dataYC, test_size=0.1, random_state=1)




 *********** Training Model for BaseC *********** 



## Phosphorous Model

In [ ]:
modelp = Sequential()
modelp.add(Dense(1000, activation='relu', input_dim = np_input))
modelp.add(Dense(np_output))
opti=k.optimizers.Adam(lr=0.001)
modelp.compile(loss="mse",optimizer=opti,metrics=['accuracy'])
history=modelp.fit(X_train_normalized,yp_train,batch_size=5,epochs=2000,shuffle=True,validation_split=0.2,verbose=0)
ep=history.epoch
hp=pd.DataFrame(history.history)



/opt/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
hp['accuracy']

0       0.116883
1       0.000000
2       0.000000
3       0.000000
4       0.000000
          ...   
1995    0.493506
1996    0.454545
1997    0.454545
1998    0.376623
1999    0.467532
Name: accuracy, Length: 2000, dtype: float64

## Sugar Model

In [ ]:
models = Sequential()
models.add(Dense(1000, activation='relu', input_dim = ns_input))
models.add(Dense(ns_output))
opti=k.optimizers.Adam(lr=0.001)
models.compile(loss="mse",optimizer=opti,metrics=['accuracy'])
history=models.fit(Xs_train,ys_train,batch_size=5,epochs=2000,shuffle=True,validation_split=0.2,verbose=0)
es=history.epoch
hs=pd.DataFrame(history.history)

#yp_pred = modelp.predict(Xp_test, verbose=0)
#print(yp_test,yp_pred)




In [ ]:
hs["accuracy"]

0       0.000000
1       0.341772
2       0.379747
3       0.063291
4       0.202532
          ...   
1995    0.417722
1996    0.379747
1997    0.417722
1998    0.341772
1999    0.341772
Name: accuracy, Length: 2000, dtype: float64

##  BaseA training model

In [ ]:
modelA = Sequential()
modelA.add(Dense(1000, activation='relu', input_dim = nA_input))
modelA.add(Dense(nA_output))
opti=k.optimizers.Adam(lr=0.001)
modelA.compile(loss="mse",optimizer=opti,metrics=['accuracy'])
history=modelA.fit(XA_train,yA_train,batch_size=5,epochs=2000,shuffle=True,validation_split=0.2,verbose=0)
eA=history.epoch
h=pd.DataFrame(history.history)

#yp_pred = modelp.predict(Xp_test, verbose=0)
#print(yp_test,yp_pred)




In [ ]:
h["accuracy"]

0       0.0000
1       0.0625
2       0.1250
3       0.1875
4       0.1875
         ...  
1995    0.1250
1996    0.1250
1997    0.1250
1998    0.1250
1999    0.1875
Name: accuracy, Length: 2000, dtype: float64

## BaseT training model

In [ ]:
modelT = Sequential()
modelT.add(Dense(1000, activation='relu', input_dim = nT_input))
modelT.add(Dense(nT_output))
opti=k.optimizers.Adam(lr=0.001)
modelT.compile(loss="mse",optimizer=opti,metrics=['accuracy'])
history=modelT.fit(XT_train,yT_train,batch_size=5,epochs=2000,shuffle=True,validation_split=0.2,verbose=0)
eT=history.epoch
ht=pd.DataFrame(history.history)

#yp_pred = modelp.predict(Xp_test, verbose=0)
#print(yp_test,yp_pred)




In [ ]:
ht["accuracy"]

0       0.0000
1       0.0000
2       0.0000
3       0.0000
4       0.0000
         ...  
1995    0.0625
1996    0.1875
1997    0.0625
1998    0.1875
1999    0.0625
Name: accuracy, Length: 2000, dtype: float64

## BaseG training model

In [ ]:
modelG = Sequential()
modelG.add(Dense(1000, activation='relu', input_dim = nG_input))
modelG.add(Dense(nG_output))
opti=k.optimizers.Adam(lr=0.001)
modelG.compile(loss="mse",optimizer=opti,metrics=['accuracy'])
history=modelG.fit(XG_train,yG_train,batch_size=5,epochs=2000,shuffle=True,validation_split=0.2,verbose=0)
eG=history.epoch
hg=pd.DataFrame(history.history)

#yp_pred = modelp.predict(Xp_test, verbose=0)
#print(yp_test,yp_pred)




In [ ]:
hg["accuracy"]

0       0.000000
1       0.000000
2       0.045455
3       0.045455
4       0.045455
          ...   
1995    0.181818
1996    0.227273
1997    0.181818
1998    0.227273
1999    0.272727
Name: accuracy, Length: 2000, dtype: float64

## BaseC training model

In [ ]:
modelC = Sequential()
modelC.add(Dense(1000, activation='relu', input_dim = nC_input))
modelC.add(Dense(nC_output))
opti=k.optimizers.Adam(lr=0.001)
modelC.compile(loss="mse",optimizer=opti,metrics=['accuracy'])
history=modelC.fit(XC_train,yC_train,batch_size=5,epochs=2000,shuffle=True,validation_split=0.2,verbose=0)
eC=history.epoch
hc=pd.DataFrame(history.history)

#yp_pred = modelp.predict(Xp_test, verbose=0)
#print(yp_test,yp_pred)




In [ ]:
hc["accuracy"]

0       0.000000
1       0.045455
2       0.090909
3       0.363636
4       0.363636
          ...   
1995    0.500000
1996    0.454545
1997    0.500000
1998    0.454545
1999    0.636364
Name: accuracy, Length: 2000, dtype: float64

In [ ]:
def data_input(cg_input,n_res,dist_res,angle):
    tmp=[]
    for k in range(n_res):
        tmp.append(float(cg_input[6]));tmp.append(dist_res[k])
        tmp.append(float(cg_input[7]));tmp.append(dist_res[k])
        tmp.append(float(cg_input[8]));tmp.append(dist_res[k])
        tmp.append(angle[k])
    return tmp

def phos_all_atom(p_input,p_output,n_res,atno,f):
  atmnm=["P","O1P","O2P","O5'"]
  for i in range(n_res):
    atno+=1
    term0 = p_input[0].ljust(6)#atom#6s
    term1 = str(atno).rjust(5)#aomnum#5d
    term2 = atmnm[i].rjust(4)#atomname$#4s
    term3 = p_input[3].ljust(3)#resname#1s
    term4 = p_input[4].rjust(1) #Astring
    term5 = p_input[5].rjust(4) #resnum
    term6 = str('%8.3f' % (p_output[3*i])).rjust(8) #x
    term7 = str('%8.3f' % (p_output[3*i+1])).rjust(8) #y
    term8 = str('%8.3f' % (p_output[3*i+2])).rjust(8) #z
    f.write("%s%s %s %s %s%s    %s%s%s\n"% (term0,term1,term2,term3,term4,term5,term6,term7,term8))

def sugar_all_atom(s_input,s_output,n_res,atno,f):
  atmnm=["C5'","C4'","O4'","C1'","C3'","C2'"]
  for i in range(n_res):
    atno+=1
    term0 = s_input[0].ljust(6)#atom#6s
    term1 = str(atno).rjust(5)#aomnum#5d
    term2 = atmnm[i].rjust(4)#atomname$#4s
    term3 = s_input[3].ljust(3)#resname#1s
    term4 = s_input[4].rjust(1) #Astring
    term5 = s_input[5].rjust(4) #resnum
    term6 = str('%8.3f' % (s_output[3*i])).rjust(8) #x
    term7 = str('%8.3f' % (s_output[3*i+1])).rjust(8) #y
    term8 = str('%8.3f' % (s_output[3*i+2])).rjust(8) #z
    f.write("%s%s %s %s %s%s    %s%s%s\n"% (term0,term1,term2,term3,term4,term5,term6,term7,term8))

def base_all_atom(b_input,b_output,n_res,atno,f):
  if(b_input[3]=="DA"):
    atmnm=["N9","C8","N7","C5","C6","N6","N1","C2","N3","C4"]
    for i in range(n_res):
        atno+=1
        term0 = b_input[0].ljust(6)#atom#6s
        term1 = str(atno).rjust(5)#aomnum#5d
        term2 = atmnm[i].rjust(4)#atomname$#4s
        term3 = b_input[3].ljust(3)#resname#1s
        term4 = b_input[4].rjust(1) #Astring
        term5 = b_input[5].rjust(4) #resnum
        term6 = str('%8.3f' % (b_output[3*i])).rjust(8) #x
        term7 = str('%8.3f' % (b_output[3*i+1])).rjust(8) #y
        term8 = str('%8.3f' % (b_output[3*i+2])).rjust(8) #z
        f.write("%s%s %s %s %s%s    %s%s%s\n"% (term0,term1,term2,term3,term4,term5,term6,term7,term8))
  elif(b_input[3]=="DT"):
    atmnm=["N1","C6","C5","C4","O4","N3","C2","O2"]
    for i in range(n_res):
        atno+=1
        term0 = b_input[0].ljust(6)#atom#6s
        term1 = str(atno).rjust(5)#aomnum#5d
        term2 = atmnm[i].rjust(4)#atomname$#4s
        term3 = b_input[3].ljust(3)#resname#1s
        term4 = b_input[4].rjust(1) #Astring
        term5 = b_input[5].rjust(4) #resnum
        term6 = str('%8.3f' % (b_output[3*i])).rjust(8) #x
        term7 = str('%8.3f' % (b_output[3*i+1])).rjust(8) #y
        term8 = str('%8.3f' % (b_output[3*i+2])).rjust(8) #z
        f.write("%s%s %s %s %s%s    %s%s%s\n"% (term0,term1,term2,term3,term4,term5,term6,term7,term8))
  elif(b_input[3]=="DG"):
    atmnm=["N9","C8","N7","C5","C6","O6","N1","C2","N2","N3","C4"]
    for i in range(n_res):
        atno+=1
        term0 = b_input[0].ljust(6)#atom#6s
        term1 = str(atno).rjust(5)#aomnum#5d
        term2 = atmnm[i].rjust(4)#atomname$#4s
        term3 = b_input[3].ljust(3)#resname#1s
        term4 = b_input[4].rjust(1) #Astring
        term5 = b_input[5].rjust(4) #resnum
        term6 = str('%8.3f' % (b_output[3*i])).rjust(8) #x
        term7 = str('%8.3f' % (b_output[3*i+1])).rjust(8) #y
        term8 = str('%8.3f' % (b_output[3*i+2])).rjust(8) #z
        f.write("%s%s %s %s %s%s    %s%s%s\n"% (term0,term1,term2,term3,term4,term5,term6,term7,term8))
  elif(b_input[3]=="DC"):
    atmnm=["N1","C6","C5","C4","N4","N3","C2","O2"]
    for i in range(n_res):
        atno+=1
        term0 = b_input[0].ljust(6)#atom#6s
        term1 = str(atno).rjust(5)#aomnum#5d
        term2 = atmnm[i].rjust(4)#atomname$#4s
        term3 = b_input[3].ljust(3)#resname#1s
        term4 = b_input[4].rjust(1) #Astring
        term5 = b_input[5].rjust(4) #resnum
        term6 = str('%8.3f' % (b_output[3*i])).rjust(8) #x
        term7 = str('%8.3f' % (b_output[3*i+1])).rjust(8) #y
        term8 = str('%8.3f' % (b_output[3*i+2])).rjust(8) #z
        f.write("%s%s %s %s %s%s    %s%s%s\n"% (term0,term1,term2,term3,term4,term5,term6,term7,term8))






In [ ]:
data_cg=[]
with open("coarse_grain.pdb") as file:
    for line in file:
        if (line[0:4] == "ATOM"):
            data_cg.append(line.split())


f1=open("All-Atom_anglemodel_n.pdb",'w')
atom_no=0;coun=0
for i in range(len(data_cg)):
    if(data_cg[i][2]=='P'):
        cg_input=data_cg[i]
        model_input=[data_input(cg_input,n_phos,dist_p,final_angle_p)]
        model_output=modelp.predict(model_input)[0]
        phos_all_atom(cg_input,model_output,n_phos,atom_no,f1)
        atom_no+=n_phos
    if(data_cg[i][2]=='S'):
        #print(data_cg[i])
        coun+=1
        cg_input=data_cg[i]
        model_input=[data_input(cg_input,n_sugar,dist_S,final_angle_s)]
        model_output=models.predict(model_input)[0]
        sugar_all_atom(cg_input,model_output,n_sugar,atom_no,f1)
        atom_no+=n_sugar
    if(data_cg[i][2]=='B'):
        cg_input=data_cg[i]
        if(cg_input[3]=="DA"):
            model_input=[data_input(cg_input,n_baseA,dist_BA,final_angle_baseA)]
            model_output=modelA.predict(model_input)[0]
            base_all_atom(cg_input,model_output,n_baseA,atom_no,f1)
            atom_no+=n_baseA
        if(cg_input[3]=="DT"):
            model_input=[data_input(cg_input,n_baseT,dist_BT,final_angle_baseT)]
            model_output=modelT.predict(model_input)[0]
            base_all_atom(cg_input,model_output,n_baseT,atom_no,f1)
            atom_no+=n_baseT
        if(cg_input[3]=="DG"):
            model_input=[data_input(cg_input,n_baseG,dist_BG,final_angle_baseG)]
            model_output=modelG.predict(model_input)[0]
            base_all_atom(cg_input,model_output,n_baseG,atom_no,f1)
            atom_no+=n_baseG
        if(cg_input[3]=="DC"):
            model_input=[data_input(cg_input,n_baseC,dist_BC,final_angle_baseC)]
            model_output=modelC.predict(model_input)[0]
            base_all_atom(cg_input,model_output,n_baseC,atom_no,f1)
            atom_no+=n_baseC

1/1 [==============================] - 0s 47ms/step


1/1 [==============================] - 0s 47ms/step


In [ ]:
one_p[0]

array([[ 37.17      ,   0.334947  ,  38.08      ,   0.334947  ,
        124.13      ,   0.334947  ,  87.78503187,  37.17      ,
          1.44695243,  38.08      ,   1.44695243, 124.13      ,
          1.44695243, 122.58655891,  37.17      ,   1.45175314,
         38.08      ,   1.45175314, 124.13      ,   1.45175314,
        117.96015479,  37.17      ,   1.4553798 ,  38.08      ,
          1.4553798 , 124.13      ,   1.4553798 , 107.37685042]])

In [ ]:
Xp_test

array([[ 38.428     ,   0.334947  ,  27.383     ,   0.334947  ,
         47.332     ,   0.334947  ,  87.78503187,  38.428     ,
          1.44695243,  27.383     ,   1.44695243,  47.332     ,
          1.44695243, 122.58655891,  38.428     ,   1.45175314,
         27.383     ,   1.45175314,  47.332     ,   1.45175314,
        117.96015479,  38.428     ,   1.4553798 ,  27.383     ,
          1.4553798 ,  47.332     ,   1.4553798 , 107.37685042],
       [ 38.442     ,   0.334947  ,  27.4       ,   0.334947  ,
         81.565     ,   0.334947  ,  87.78503187,  38.442     ,
          1.44695243,  27.4       ,   1.44695243,  81.565     ,
          1.44695243, 122.58655891,  38.442     ,   1.45175314,
         27.4       ,   1.45175314,  81.565     ,   1.45175314,
        117.96015479,  38.442     ,   1.4553798 ,  27.4       ,
          1.4553798 ,  81.565     ,   1.4553798 , 107.37685042],
       [ 30.105     ,   0.334947  ,  20.36      ,   0.334947  ,
         74.675     ,   0.334947  ,  8

In [ ]:
model_inp = [[37.273, 0.3421120184016555, 35.227, 0.3421120184016555, 35.227, 0.3421120184016555, 87.722069277059, 37.273, 1.4508075749272553, 35.227, 1.4508075749272553, 35.227, 1.4508075749272553, 122.62876804907191, 37.273, 1.4469922008899332, 35.227, 1.4469922008899332, 35.227, 1.4469922008899332, 117.5938166368181, 37.273, 1.456064256364565, 35.227, 1.456064256364565, 35.227, 1.456064256364565, 107.6973246801287]]
inp = scaler.transform(Xp_test)
model_output=modelp.predict(X_test_normalized)




1/1 [==============================] - 0s 170ms/step


In [ ]:
model_output[:5]

array([[38.423656, 27.318056, 47.352753, 39.461742, 27.462193, 47.064075,
        38.190746, 26.79877 , 47.680862, 37.45306 , 28.19018 , 47.57752 ],
       [38.507057, 27.360949, 81.5913  , 39.51404 , 27.400745, 81.392426,
        38.135403, 26.818834, 81.75278 , 37.431293, 28.169737, 81.61052 ],
       [30.22766 , 20.397295, 74.71597 , 30.158756, 19.404303, 74.54474 ,
        29.89127 , 19.980057, 74.74436 , 30.038675, 21.794256, 74.65923 ],
       [38.81934 , 33.3617  , 84.99428 , 39.828846, 34.194378, 84.81148 ,
        38.42956 , 32.84714 , 85.18234 , 37.675434, 33.554657, 85.02715 ],
       [35.37075 , 22.13621 , 44.128395, 36.031055, 21.542185, 43.858334,
        35.15599 , 21.650282, 44.37024 , 34.72876 , 23.485092, 44.298416]],
      dtype=float32)

In [ ]:
yp_test[:5]

array([[38.42, 27.57, 47.62, 39.54, 26.66, 47.83, 38.61, 28.65, 46.63,
        37.14, 26.65, 47.25],
       [38.43, 27.54, 81.81, 39.61, 26.77, 82.2 , 38.54, 28.66, 80.85,
        37.19, 26.63, 81.4 ],
       [30.27, 20.42, 74.93, 29.63, 19.07, 75.17, 31.38, 20.45, 73.96,
        29.14, 21.5 , 74.64],
       [38.61, 33.52, 85.27, 40.07, 33.59, 85.55, 38.1 , 34.59, 84.43,
        38.23, 32.09, 84.65],
       [35.45, 22.21, 44.43, 35.61, 20.79, 44.48, 36.35, 23.05, 43.59,
        34.01, 22.56, 43.9 ]])